In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

In [4]:
import pandas as pd

In [5]:
from sklearn.multiclass import OneVsRestClassifier

In [6]:
from sklearn.metrics import accuracy_score

In [7]:
from sklearn.metrics import classification_report

In [8]:
pd.set_option('display.max_columns', None)

In [9]:
pd.set_option("max_rows", None)

In [21]:
df_train=pd.read_excel("train_agriculture.xlsx")
df_test=pd.read_excel("test_agriculture.xlsx")
df_train["dataset"]="train"
df_test["dataset"]="test"

In [22]:
df_train.shape

(4599, 11)

In [23]:
df_test.shape

(1199, 10)

In [24]:
df=pd.concat([df_train,df_test],ignore_index=True)

# Check The Correlation

In [133]:
#df.corr()

In [134]:
#df.describe()

In [135]:
#df.skew()

In [25]:
def missing_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Missing Values'})
        mz_table['Attributes'] = df.nunique()
        mz_table['Data Type'] = df.dtypes
        
        mz_table = mz_table.sort_values('Attributes', ascending=False).round(1)
        
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
               )

        return mz_table

missing_values_table(df)

Your selected dataframe has 11 columns and 5798 Rows.



,Zero Values,Missing Values,% of Total Missing Values,Attributes,Data Type
ID,0,0,0.0,5798,object
Estimated_Insects_Count,0,0,0.0,71,int64
Number_Weeks_Used,58,561,9.7,63,float64
Number_Weeks_Quit,2035,0,0.0,47,int64
Number_Doses_Week,67,0,0.0,20,int64
Pesticide_Use_Category,0,0,0.0,3,int64
Season,0,0,0.0,3,int64
Crop_Damage,3820,1199,20.7,3,float64
Crop_Type,4531,0,0.0,2,int64
Soil_Type,3060,0,0.0,2,int64


In [26]:
for col in df.columns:
    if df[col].dtype=="int64":
        print('**************************************',col,'******************************************')
        print()
        print('No. of unique attributes:',df[col].nunique())
        print()
        print('No. of unique attributes:',df[col].unique())
        print()

************************************** Estimated_Insects_Count ******************************************

No. of unique attributes: 71

No. of unique attributes: [ 188  209  257  342  448  577  731 1132 1212 1575 1785 2138 2401 2999
 3516 3895 4096  150  151  168  232  231  256  283  312  311  375  411
  410  488  489  531  626  625  677  732  789  790  851  850  915  916
  984 1056 1296 1297 1385 1478 1576 1678 1786 1898 2015 2016 2139 2267
 2402 2542 2541 2688 2687 2840 3164 3165 3336 3337 3702 3896  169 3515
 4097]

************************************** Crop_Type ******************************************

No. of unique attributes: 2

No. of unique attributes: [1 0]

************************************** Soil_Type ******************************************

No. of unique attributes: 2

No. of unique attributes: [0 1]

************************************** Pesticide_Use_Category ******************************************

No. of unique attributes: 3

No. of unique attributes: [1 

In [27]:
for col in df.columns:
    if df[col].dtype=="float64":
        print('**************************************',col,'******************************************')
        print()
        print('No. of unique attributes:',df[col].nunique())
        print()
        print('No. of unique attributes:',df[col].unique())
        print()

************************************** Number_Weeks_Used ******************************************

No. of unique attributes: 63

No. of unique attributes: [ 0. nan 26. 20. 15. 14. 19. 24.  8. 23. 25. 28. 29. 22. 21. 17. 30. 35.
 32. 27. 16. 31. 33. 34. 39. 38. 42. 40. 41. 36. 44. 37. 46. 45. 47. 43.
 50. 10. 49. 53. 48. 51. 54. 52. 55. 56. 61. 18. 57. 62. 60. 63. 66. 59.
 64. 13.  7.  5. 12.  6.  9. 58. 11. 65.]

************************************** Crop_Damage ******************************************

No. of unique attributes: 3

No. of unique attributes: [ 0.  1.  2. nan]



In [28]:
df = df.drop(['ID'],axis=1)

In [29]:
df.head()

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage,dataset
0,188,1,0,1,0,0.0,0,1,0.0,train
1,209,1,0,1,0,0.0,0,2,1.0,train
2,257,1,0,1,0,0.0,0,2,1.0,train
3,257,1,1,1,0,0.0,0,2,1.0,train
4,342,1,0,1,0,0.0,0,2,1.0,train


In [30]:
df[(df.Number_Weeks_Used.isnull())]

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage,dataset
5,448,0,1,1,0,NaN,0,2,1.0,train
6,448,0,1,1,0,NaN,0,2,1.0,train
10,1212,1,0,1,0,NaN,0,3,0.0,train
17,2401,0,1,1,0,NaN,0,1,1.0,train
35,257,0,0,3,30,NaN,0,2,0.0,train
51,411,0,0,3,30,NaN,0,2,0.0,train
59,448,0,0,3,10,NaN,0,2,1.0,train
60,448,0,0,3,30,NaN,0,3,1.0,train
65,448,0,1,3,45,NaN,0,2,0.0,train
80,531,0,0,3,20,NaN,0,1,0.0,train


In [31]:
df[(df.Number_Weeks_Used.isnull()) & (df.dataset=='train')].shape

(442, 10)

In [32]:
df[(df.Number_Weeks_Used.isnull()) & (df.dataset=='test')].shape

(119, 10)

In [33]:
df.dropna(inplace=True)

In [34]:
def missing_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Missing Values'})
        mz_table['Attributes'] = df.nunique()
        mz_table['Data Type'] = df.dtypes
        
        mz_table = mz_table.sort_values('Attributes', ascending=False).round(1)
        
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
               )

        return mz_table

missing_values_table(df)

Your selected dataframe has 10 columns and 4157 Rows.



,Zero Values,Missing Values,% of Total Missing Values,Attributes,Data Type
Estimated_Insects_Count,0,0,0.0,71,int64
Number_Weeks_Used,40,0,0.0,63,float64
Number_Weeks_Quit,1336,0,0.0,47,int64
Number_Doses_Week,40,0,0.0,20,int64
Pesticide_Use_Category,0,0,0.0,3,int64
Season,0,0,0.0,3,int64
Crop_Damage,3465,0,0.0,3,float64
Crop_Type,3220,0,0.0,2,int64
Soil_Type,2181,0,0.0,2,int64
dataset,0,0,0.0,1,object


In [36]:
#Lets seprate the train and test from df_final
df_train=df.loc[df["dataset"]=="train"]
df_test=df.loc[df["dataset"]=="test"]

In [37]:
#Lets drop  "dataset" columns from the test dataset
#Also droping "dataset" columns from the train dataset
df_train.drop(columns=["dataset"],inplace=True)
df_test.drop(columns=["dataset"],inplace=True)

In [138]:
df_test.drop(columns=["Crop_Damage"],inplace=True)

In [139]:
df_test.shape

(0, 8)

# Assign THe Values (Input & Output)

In [38]:
df.columns

Index(['Estimated_Insects_Count', 'Crop_Type', 'Soil_Type',
       'Pesticide_Use_Category', 'Number_Doses_Week', 'Number_Weeks_Used',
       'Number_Weeks_Quit', 'Season', 'Crop_Damage', 'dataset'],
      dtype='object')

In [100]:
df_x = df_train.drop(['Crop_Damage'],axis=1)
y = df_train['Crop_Damage']

In [101]:
df_x.shape

(4157, 8)

In [102]:
y.shape

(4157,)

In [103]:
y.nunique()

3

# Evaluation

In [117]:
#Lets do standard scaling of the input variable
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
def maxf1_score(clf,df_x,y):
    maxf=0
    rs=0
    for r_state in range(42,100):
        x_train,x_test,y_train,y_test=train_test_split(df_x, y,random_state = r_state,test_size=0.20,stratify=y)
        x_train, y_train = SMOTE().fit_resample(x_train, y_train)
           
        clf.fit(x_train,y_train)
        y_pred=clf.predict(x_test)
        tmp=f1_score(y_test,y_pred, average='micro')
        print("random state :",r_state," and f1 score: ",tmp)
        if tmp>maxf:
            maxf=tmp
            rs=r_state
    print()
    print("x_train :",x_train.shape[0]," y_train :",y_train.shape[0])
    print()
    print(" y_train :")
    print(y_train.value_counts())
    print()
    print(" y_test :")
    print()
    print(y_test.value_counts())
    print("x_test :",x_test.shape," y_test :",y_test.shape)
    print("maximum f1_score is at random state :",rs," and it is :",maxf)

# Decision Tree

In [118]:
#lets use decision tree classifier
from sklearn.tree import DecisionTreeClassifier
dc=DecisionTreeClassifier()
maxf1_score(dc,df_x,y)

random state : 42  and f1 score:  0.6490384615384616
random state : 43  and f1 score:  0.6802884615384616
random state : 44  and f1 score:  0.6923076923076923
random state : 45  and f1 score:  0.6574519230769231
random state : 46  and f1 score:  0.671875
random state : 47  and f1 score:  0.6550480769230769
random state : 48  and f1 score:  0.6778846153846154
random state : 49  and f1 score:  0.6766826923076923
random state : 50  and f1 score:  0.7079326923076922
random state : 51  and f1 score:  0.6862980769230769
random state : 52  and f1 score:  0.6754807692307693
random state : 53  and f1 score:  0.6598557692307693
random state : 54  and f1 score:  0.6346153846153846
random state : 55  and f1 score:  0.6658653846153846
random state : 56  and f1 score:  0.6658653846153846
random state : 57  and f1 score:  0.6838942307692307
random state : 58  and f1 score:  0.6682692307692307
random state : 59  and f1 score:  0.6670673076923077
random state : 60  and f1 score:  0.6670673076923077
ran

In [120]:
x_train, x_test, y_train, y_test = train_test_split(df_x, y,random_state = 65,test_size=0.20)
y_pred_dc = dc.predict(x_test)

In [121]:
target_names = ['class 0', 'class 1', 'class 2']

In [122]:
y_test.nunique()

3

In [123]:
y_test.value_counts()

0.0    681
1.0    138
2.0     13
Name: Crop_Damage, dtype: int64

In [124]:
print(classification_report(y_test, y_pred_dc, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.97      0.95      0.96       681
     class 1       0.81      0.84      0.83       138
     class 2       0.36      0.69      0.47        13

    accuracy                           0.93       832
   macro avg       0.71      0.83      0.75       832
weighted avg       0.94      0.93      0.93       832



# Random Forest

In [49]:
#lets chcek random forest also
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
parameters={"n_estimators":[10,100,500]}
rf_clf=RandomForestClassifier()
clf = GridSearchCV(rf_clf, parameters, cv=10,scoring="accuracy")
clf.fit(df_x,y)
clf.best_params_

{'n_estimators': 10}

In [125]:
rf_clf=RandomForestClassifier(n_estimators=10)
maxf1_score(rf_clf,df_x,y)

random state : 42  and f1 score:  0.7211538461538461
random state : 43  and f1 score:  0.7271634615384615
random state : 44  and f1 score:  0.7415865384615384
random state : 45  and f1 score:  0.7403846153846154
random state : 46  and f1 score:  0.7223557692307693
random state : 47  and f1 score:  0.7403846153846154
random state : 48  and f1 score:  0.7331730769230769
random state : 49  and f1 score:  0.7235576923076923
random state : 50  and f1 score:  0.75
random state : 51  and f1 score:  0.7620192307692307
random state : 52  and f1 score:  0.7451923076923076
random state : 53  and f1 score:  0.7247596153846154
random state : 54  and f1 score:  0.7271634615384615
random state : 55  and f1 score:  0.7307692307692306
random state : 56  and f1 score:  0.7223557692307693
random state : 57  and f1 score:  0.734375
random state : 58  and f1 score:  0.7403846153846154
random state : 59  and f1 score:  0.7307692307692306
random state : 60  and f1 score:  0.719951923076923
random state : 61 

In [126]:
x_train, x_test, y_train, y_test = train_test_split(df_x, y,random_state = 80,test_size=0.20)
y_pred_rf = rf_clf.predict(x_test)

In [127]:
target_names = ['class 0', 'class 1', 'class 2']

In [128]:
y_test.nunique()

3

In [129]:
y_test.value_counts()

0.0    688
1.0    124
2.0     20
Name: Crop_Damage, dtype: int64

In [84]:
print(classification_report(y_test, y_pred_rf, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.96      0.96      0.96       688
     class 1       0.76      0.77      0.77       124
     class 2       0.61      0.70      0.65        20

    accuracy                           0.92       832
   macro avg       0.78      0.81      0.79       832
weighted avg       0.92      0.92      0.92       832



In [130]:
#lets chcek KNN
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings("ignore")
kc=KNeighborsClassifier()
neighbors={"n_neighbors":range(1,30)}
clf = GridSearchCV(kc, neighbors, cv=5,scoring="accuracy")
clf.fit(df_x,y)
clf.best_params_

{'n_neighbors': 29}

In [131]:
kc=KNeighborsClassifier(n_neighbors=29)
maxf1_score(kc,df_x,y)

random state : 42  and f1 score:  0.5480769230769231
random state : 43  and f1 score:  0.5600961538461539
random state : 44  and f1 score:  0.5612980769230769
random state : 45  and f1 score:  0.5276442307692307
random state : 46  and f1 score:  0.5432692307692307
random state : 47  and f1 score:  0.4795673076923077
random state : 48  and f1 score:  0.5300480769230769
random state : 49  and f1 score:  0.5336538461538461
random state : 50  and f1 score:  0.5300480769230769
random state : 51  and f1 score:  0.5649038461538461
random state : 52  and f1 score:  0.5192307692307693
random state : 53  and f1 score:  0.5240384615384616
random state : 54  and f1 score:  0.5528846153846154
random state : 55  and f1 score:  0.5372596153846154
random state : 56  and f1 score:  0.5360576923076923
random state : 57  and f1 score:  0.5264423076923077
random state : 58  and f1 score:  0.5588942307692307
random state : 59  and f1 score:  0.5444711538461539
random state : 60  and f1 score:  0.5276442307

In [132]:
x_train, x_test, y_train, y_test = train_test_split(df_x, y,random_state = 51,test_size=0.20)
y_pred_kc = kc.predict(x_test)

In [133]:
target_names = ['class 0', 'class 1', 'class 2']

y_test.nunique()

3

In [134]:
y_test.value_counts()

0.0    691
1.0    122
2.0     19
Name: Crop_Damage, dtype: int64

In [135]:
print(classification_report(y_test, y_pred_kc, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.94      0.61      0.74       691
     class 1       0.30      0.60      0.40       122
     class 2       0.09      0.68      0.16        19

    accuracy                           0.61       832
   macro avg       0.44      0.63      0.43       832
weighted avg       0.83      0.61      0.67       832



# Prediction

In [140]:
y_pred_final = rf_clf.predict(x_test)

In [141]:
#lets make the dataframe for price_pred
prediction=pd.DataFrame(y_pred_final,columns=["Crop_Damage"])

In [142]:
#Lets save the submission to csv
prediction.to_csv("Crop_Damage_Predictions.csv",index=False)